In [ ]:
import multiprocessing
import warnings
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML

from lib.comstock_processor import ComStockProcessor

dataset_path = Path().resolve().cwd() / "datasets"
print(f"Dataset path: {dataset_path}")

data_path = dataset_path / "comstock"
print(f"Data path: {data_path}")

time_series_data_path = data_path / "time_series_data"
print(f"Time Series Data path: {time_series_data_path}")

figures_path = Path().resolve() / "figures"
for path in [figures_path, time_series_data_path]:
    if not path.exists():
        path.mkdir(parents=True)
print(f"Figures path: {figures_path}")

# ignore SettingWithCopyWarning
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

# Allow autoreload as we develop dependent packages in parallel
%load_ext autoreload
%autoreload 2

# ComStock

### Pull down ComStock metadata

The data will be saved into the ComStock datasets subfolder. It takes a while to run (10ish minutes) but will only download the file if it does not already exist.

### Read in ComStock data

In [ ]:
# Download and convert the data file to a dataframe.
processor = ComStockProcessor(state="CA", county_name="All", building_type="All", upgrade="0", base_dir=data_path)
df_all = processor.process_metadata(save_dir=data_path)
display(df_all)

In [ ]:
# sample the df_all data. First, group all the buildings by building_type, county_name, and heating_fuel.
# Set seed to 42, to make it reproducible for us all
df_all_sample = df_all.groupby(["in.comstock_building_type", "in.county_name", "in.heating_fuel"]).apply(
    lambda x: x.sample(1, random_state=42)
)

# reflatten the data
df_all_sample = df_all_sample.reset_index(drop=True)
display(df_all_sample)

# plot the parameter spaces: in.building_type, county_name, heating fuel
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
df_all_sample.plot.scatter(x="in.county_name", y="in.comstock_building_type", c="in.sqft", cmap="viridis", ax=ax)
plt.xticks(rotation=90)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
df_all_sample.plot.scatter(x="in.heating_fuel", y="in.comstock_building_type", c="in.sqft", cmap="viridis", ax=ax)
plt.xticks(rotation=90)

# get list of IDs from the sample,
download_timeseries_id = []

## Simplify data fields available and write simplified version to csv for analysis

In [ ]:
df_sub = df_all[
    [
        "bldg_id",
        "in.sqft",
        "in.state",
        "in.county_name",
        "in.comstock_building_type",
        "in.building_subtype",
        "in.heating_fuel",
        "out.site_energy.total.energy_consumption_intensity",
        "out.district_cooling.total.energy_consumption",
        "out.district_heating.total.energy_consumption",
        "out.electricity.total.energy_consumption",
        "out.natural_gas.total.energy_consumption",
        "out.other_fuel.total.energy_consumption",
        "out.utility_bills.electricity_bill_max..usd",
        "out.utility_bills.electricity_bill_mean..usd",
        "out.utility_bills.electricity_bill_median..usd",
        "out.utility_bills.electricity_bill_min..usd",
        "out.utility_bills.electricity_bill_number_of_rates..usd",
    ]
]

# save to csv
df_sub.to_csv(data_path / "CA-All-All-0-metadata_simplified.csv", index=False)

# Download time series data

In [ ]:
# Only download 101 buildings for sampling purposes and testing this notebook/repo
timeseries_to_download = df_sub.head(10)
print(f"Downloading time series data for {len(timeseries_to_download)} buildings into {time_series_data_path}")

time_series_data_file_paths, building_ids = processor.process_building_time_series(
    data_frame=timeseries_to_download, save_dir=time_series_data_path
)

In [ ]:
# post process all the files and build up the combined data file, this can take about 1 minute to run (with 10 cores)

# if the data file exists, then just read that, otherwise, recreate the postprocessed file.
num_workers = max(1, multiprocessing.cpu_count() - 1)

all_sampled_timeseries_filename = time_series_data_path / "all_sampled_buildings-upgrade-0.parquet"

if all_sampled_timeseries_filename.exists():
    print("Loading in the sampled timeseries data from disk, delete the file if you need to recreate it")
    all_ts_df = pd.read_parquet(all_sampled_timeseries_filename)
else:

    def read_and_process_file(args) -> pd.DataFrame:
        """Method to read and process input file so that this can be used in parallel"""
        data_file, building_id = args
        # only read a copy of the columns in the file that we need, extend as needed.
        tdf = pd.read_parquet(
            data_file,
            columns=[
                "timestamp",
                "out.electricity.total.energy_consumption",
                "out.natural_gas.total.energy_consumption",
            ],
        )
        tdf["bldg_id"] = building_id
        return tdf

    # parallel process everything
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        all_dfs = list(executor.map(read_and_process_file, zip(time_series_data_file_paths, building_ids)))

    # concatenate the dataframes that were read in parallel.
    all_ts_df = pd.concat(all_dfs, ignore_index=True)
    # move the "bldg_id" column to the beginning
    all_ts_df = all_ts_df[["bldg_id"] + [col for col in all_ts_df.columns if col != "bldg_id"]]
    all_ts_df.to_parquet(all_sampled_timeseries_filename)

display(all_ts_df)

# Data Exploration Section

In [ ]:
# get the dimensions
print(df_sub.shape)
# show all states
print(df_sub["in.state"].unique())
# show all the building types
print(df_sub["in.comstock_building_type"].unique())

In [ ]:
# save the names of the fields to a list
fields = df_all.columns
with open(data_path / "comstock_metadata_fields.txt", "w") as f:
    f.write("\n".join(fields))
    # write blank line at end
    f.write("\n")

In [ ]:
# only buildings in CA
df_ca = df_sub[df_sub["in.state"].str.contains("CA")]
print(f"all: {df_sub.shape}")
print(f"cz7: {df_ca.shape}")
for to_display in ["in.building_subtype", "in.comstock_building_type"]:
    df_to_show = df_ca[to_display].value_counts()
    df_to_show = df_to_show.reset_index()
    display(HTML(df_to_show.to_html(index=False, border=1)))

In [ ]:
county_counts = df_ca["in.county_name"].value_counts()
# make it a dataframe
county_counts = county_counts.reset_index()
# in the in.county_name remove CA, and County
county_counts["in.county_name"] = county_counts["in.county_name"].str.replace("CA,", "").str.replace("County", "")
# and trim the remainder
county_counts["in.county_name"] = county_counts["in.county_name"].str.strip()

# display the counts nicely, but with no index
display(HTML(county_counts.to_html(index=False, border=1)))

In [ ]:
# Load a built-in dataset of US counties (you need geopandas installed)
counties_gdf = gpd.read_file("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json")

# Filter for California counties (FIPS state code for CA is '06')
california_counties = counties_gdf[counties_gdf["STATE"] == "06"]

# convert county_counts to a dictionary of format {county_name: ["a", "b", "c"], count: [1, 2, 3]}
county_mapping = dict(zip(county_counts["in.county_name"], county_counts["count"]))
california_counties["count"] = california_counties["NAME"].map(county_mapping)

# Plot the map on a logscale
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
california_counties.plot(
    column="count",
    cmap="Oranges",
    legend=True,
    missing_kwds={"color": "lightgrey"},
    ax=ax,
)
plt.title("Count of ComStock Buildings in CA", fontsize=16)
plt.show()

# create another plot but without Los Angeles
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
california_counties[california_counties["NAME"] != "Los Angeles"].plot(
    column="count",
    cmap="Oranges",
    legend=True,
    missing_kwds={"color": "lightgrey"},
    ax=ax,
)
plt.title("Count of ComStock Buildings in CA (without LA)", fontsize=16)

In [ ]:
# create a pie chart of the total energy in kwh of district cooling, district heating, electricity, natural gas
meters = [
    "out.district_cooling.total.energy_consumption",
    "out.district_heating.total.energy_consumption",
    "out.electricity.total.energy_consumption",
    "out.natural_gas.total.energy_consumption",
    "out.other_fuel.total.energy_consumption",
]

# create a new dataframe with the sum of each meter
df_meters = df_ca[meters].sum()
print(df_meters)
# rename the columns to be shorter, electricity, natural gas, district heating,
# district cooling, and other fuel
df_meters = df_meters.rename(
    {
        "out.district_cooling.total.energy_consumption": "District Cooling",
        "out.district_heating.total.energy_consumption": "District Heating",
        "out.electricity.total.energy_consumption": "Electricity",
        "out.natural_gas.total.energy_consumption": "Natural Gas",
        "out.other_fuel.total.energy_consumption": "Other Fuel",
    }
)

# plot the meter totals as a pie chart
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.pie(df_meters, labels=df_meters.index, autopct="%1.1f%%")
plt.title("ComStock - Energy Consumption by Meter Type", fontsize=16)

# now show the heating fuel type by count
df_heating = df_ca["in.heating_fuel"].value_counts()
df_heating = df_heating.reset_index()
display(HTML(df_heating.to_html(index=False, border=1)))
# make a pie chart of count of heating fuels
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.pie(df_heating["count"], labels=df_heating["in.heating_fuel"], autopct="%1.1f%%")
plt.title("ComStock - Heating Fuel Type by Count", fontsize=16)